# Diff in Diff Regression by Clarissa

In [21]:
import pandas as pd

df = pd.read_csv("/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/20_analysis/big_merge.csv")
df_clean = df.dropna(axis=0).copy()
df.sample(3)

,Unnamed: 0,Unnamed: 0.1,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment,year
4247,375,375,female,Narowal,urban,514,459,0.892996,2014
4272,400,400,female,Sahiwal,rural,62,56,0.903226,2014
2985,41,41,male,Faisalabad,urban,984,851,0.864837,2012


## Aggregated

In [22]:
# Treatment and pre-post variables

df_clean.loc[(df_clean["year"] >= 2010), "post_2009"] = 1
df_clean.loc[(df_clean["year"] < 2007), "post_2009"] = 0

df_clean["Treated"] = 0
df_clean.loc[(df_clean["sex"] == "female"), "Treated"] = 1

In [23]:
# Divide into urban/rural

df_urban = df_clean.loc[df_clean["region"] == "urban"].copy()
df_rural = df_clean.loc[df_clean["region"] == "rural"].copy()
df_female = df_clean.loc[df_clean["sex"] == "female"].copy()

In [24]:
df_rural.sample(3)

,Unnamed: 0,Unnamed: 0.1,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment,year,post_2009,Treated
2186,154,154,male,Narowal,rural,596,504,0.845638,2010,1.0,0
2074,42,42,male,Dadu,rural,919,631,0.686616,2010,1.0,0
4074,202,202,male,Toba Tek Singh,rural,120,109,0.908333,2014,1.0,0


In [25]:
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

mod_aggregated = smf.ols('rate_enrollment ~ C(post_2009) * C(Treated)', df_clean).fit()
mod_aggregated.get_robustcov_results(cov_type="HC3").summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        rate_enrollment   R-squared:                       0.131
Model:                            OLS   Adj. R-squared:                  0.130
Method:                 Least Squares   F-statistic:                     195.3
Date:                Tue, 05 Apr 2022   Prob (F-statistic):          4.15e-118
Time:                        19:23:47   Log-Likelihood:                 925.84
No. Observations:                3944   AIC:                            -1844.
Df Residuals:                    3940   BIC:                            -1819.
Df Model:                           3                                         
Covariance Type:                  HC3                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               0.6850      0.009     78.824      0.000       0.668       0.702
C(post_2009)[T.1.0]                     0.0425      0.009      4.514      0.000       0.024       0.061
C(Treated)[T.1]                        -0.1726      0.015    -11.313      0.000      -0.203      -0.143
C(post_2009)[T.1.0]:C(Treated)[T.1]     0.0397      0.017      2.385      0.017       0.007       0.072
==============================================================================
Omnibus:                      133.934   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              137.788
Skew:                          -0.432   Prob(JB):                     1.20e-30
Kurtosis:                       2.696   Cond. No.                         12.0
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

When we look at all rural and urban areas of all districts in Pakistan, the enrollment rate increases for girls *4 percentage points* more than for boys.
We also see that for boys, the rate of enrollment in schools declines after the years we are studying.

**LIMITATIONS:**
* We are only observing 3 year prior the Taliban campaign
* We are fitting a linear regression model to a relationship that may not be linear. We can consider the effect the taliban campaign is multifacetic and has many long term and short term effect that interact...
* Would be nice to cluster by region(rural vs urban) bu we dont have complete data of urban and rural 

## Rural Districts only

In [26]:
# RURAL

mod_rural = smf.ols('rate_enrollment ~ C(post_2009) * C(Treated)', df_rural).fit()
mod_rural.get_robustcov_results(cov_type="HC3").summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        rate_enrollment   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     139.9
Date:                Tue, 05 Apr 2022   Prob (F-statistic):           6.83e-83
Time:                        19:23:47   Log-Likelihood:                 393.14
No. Observations:                2178   AIC:                            -778.3
Df Residuals:                    2174   BIC:                            -755.5
Df Model:                           3                                         
Covariance Type:                  HC3                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               0.6644      0.010     66.734      0.000       0.645       0.684
C(post_2009)[T.1.0]                     0.0437      0.011      3.822      0.000       0.021       0.066
C(Treated)[T.1]                        -0.1966      0.017    -11.708      0.000      -0.230      -0.164
C(post_2009)[T.1.0]:C(Treated)[T.1]     0.0418      0.020      2.131      0.033       0.003       0.080
==============================================================================
Omnibus:                       61.446   Durbin-Watson:                   1.661
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.308
Skew:                          -0.207   Prob(JB):                     1.77e-09
Kurtosis:                       2.478   Cond. No.                         9.76
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

Not too different at least in terms of the regression. Although, noticed a few things:
* Larger standard error

## Cities controlled by terrorist in 2009 vs not

In [27]:
taliabn_dominance = [
    "South Waziristan",
    "North Waziristan",
    "Orakzai",
    "Kurram",
    "Khyber",
    "Mohmand",
    "Bajur",
    "Darra Adamkhel",
    "Swat",
    "Upper Dir",
    "Lower Dir",
    "Bannu",
    "Lakki Marwat",
    "Tank",
    "Peshawar",
    "Dera Ismail Khan",
    "Mardan",
    "Charsadda",
    "Kohat",
]

In [28]:
# New treatment variables

df_female["Treated"] = 0
df_female.loc[(df_female["subprovince"].isin(taliabn_dominance)), "Treated"] = 1
df_female.sample(5)

,Unnamed: 0,Unnamed: 0.1,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment,year,post_2009,Treated
3342,398,398,female,Rajanpur,rural,511,211,0.412916,2012,1.0,0
4594,266,266,female,Charsadda,rural,82,41,0.500000,2015,1.0,1
5397,69,69,female,Jhang,rural,1508,1015,0.673077,2019,1.0,0
3842,442,442,female,Swabi,rural,162,115,0.709877,2013,1.0,0
627,239,239,female,Lahore,urban,437,332,0.759725,2005,0.0,0


In [30]:
# Taliban proximity

mod_proximity = smf.ols('rate_enrollment ~ C(post_2009) * C(Treated)', df_female).fit()
mod_proximity.get_robustcov_results(cov_type="HC3").summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        rate_enrollment   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     21.19
Date:                Tue, 05 Apr 2022   Prob (F-statistic):           1.65e-13
Time:                        19:24:06   Log-Likelihood:                 134.42
No. Observations:                1972   AIC:                            -260.8
Df Residuals:                    1968   BIC:                            -238.5
Df Model:                           3                                         
Covariance Type:                  HC3                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               0.5204      0.014     37.757      0.000       0.493       0.547
C(post_2009)[T.1.0]                     0.0749      0.015      4.976      0.000       0.045       0.104
C(Treated)[T.1]                        -0.0729      0.025     -2.869      0.004      -0.123      -0.023
C(post_2009)[T.1.0]:C(Treated)[T.1]     0.0664      0.029      2.262      0.024       0.009       0.124
==============================================================================
Omnibus:                      252.436   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               93.693
Skew:                          -0.314   Prob(JB):                     4.52e-21
Kurtosis:                       2.136   Cond. No.                         14.8
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""